In [ ]:
# %%
from shapely.geometry import Point 


pt = Point(5, 5)
pt
# %%
!pip install shapely

# %%
pt.x, pt.y

# %%
from shapely.geometry import Polygon

poly= Polygon([
    [0, 0],
    [0, 5],
    [5, 5],
    [5, 0],

])
poly

# %%
poly.area
# %%
poly.centroid

# %%
poly.intersects(pt)
# %%
poly.intersects(Point(10,20))

# %%
import pandas as pd
df = pd.read_csv(
    'geotrack.csv',
    parse_dates=['time'],
    index_col='time',

)
df= df.resample('min').mean()
# %%
df['point'] = (
    df[['lat', 'lng']]
    .apply(Point, axis=1)
)
df.head()
# %%
mid_lat, max_lat = \
    df['lat']. mean(), df['lat'].max()
mid_lng, max_lng = \
    df['lng']. mean(), df['lng'].max()
poly = Polygon([
    [mid_lat, mid_lng],
    [mid_lat, max_lng],
    [max_lat, max_lng],
    [max_lat, mid_lng],
])
poly
# %%
df[df['point'].apply(poly.intersects)]

# %%
poly.exterior.xy

# %%
import numpy as np
np.stack(poly.exterior.xy)

# %%
np.stack(poly.exterior.xy).T

# %%
import folium 
m = folium.Map(
    location=[df['lat'].mean(), df['lng'].mean()],
    zoom_start=14,
)

points = np.stack(poly.exterior.xy).T
m.add_child(folium.PolyLine(points))

m

def add_marker(row):
    loc = tuple(row[['lat', 'lng']])
    in_poly = poly.intersects(row['point'])
    marker = folium.CircleMarker(
        loc,
        radius=5,
        color='blue' if in_poly else'red',
        popup=row.name.strftime('%H:%M'),

    )
    marker.add_to(m)
df.apply(add_marker, axis=1)
m
# %%
def add_marker(row):
    loc = tuple(row[['lat', 'lng']])
    height = row['height']
    in_poly = poly.intersects(row['point'])
    marker = folium.CircleMarker(
        loc,
        radius=5,
        color='blue' if  height<100 else'red',
        popup=row.name.strftime('%H:%M'),

    )
    marker.add_to(m)
df.apply(add_marker, axis=1)
m